In [144]:
import pandas as pd

In [145]:
df = pd.read_csv('lista_zawiadomień.csv', encoding='utf-8-sig', sep=";")

# Handle missing values in Nr zlec., Opis i Priorytet
df['Nr zlec.'] = df['Nr zlec.'].fillna('Brak danych')
df['Opis'] = df['Opis'].fillna('Brak opisu')
df['Priorytet'] = df['Priorytet'].fillna('Nieokreślony')

# Delete rows with NaN values in Oznaczenie (name of the machine)
df = df.dropna(subset=['Oznaczenie'])

# Delete not needed columns: Utworzone przez, Opis, Status systemu and Nr zlec.
df = df.drop(columns=['Utworzone przez', 'Opis', 'Status systemu', "Nr zlec."])

# Change type of the column Data zawiadom. to date
df['Data zawiadom.'] = pd.to_datetime(df['Data zawiadom.'], format='%d.%m.%Y', errors='coerce')

df.to_csv("cleaned_lista_zawiadomień.csv", index=False, encoding="utf-8")
df

,Rodzaj zawiad.,Zawiadomienie,Data zawiadom.,Oznaczenie,Priorytet
0,1P,12708097,2025-06-30,1-Zespół pomp próżniowych BUSCH,4-niski
1,1P,12707957,2025-06-30,1-Trans. Sandwick za walcami 64M,Nieokreślony
2,1P,12707755,2025-06-30,1-Zbiornik olejowy 1m^3,Nieokreślony
3,1P,12707745,2025-06-30,1-Piec wafla górnego KB2,Nieokreślony
4,PQ,12707726,2025-06-30,1-Installations Other,3-średni
...,...,...,...,...,...
160648,PM,10143679,2014-04-01,2-Kinder Joy lini-Nalewarka goccia,Nieokreślony
160649,PM,10143678,2014-04-01,1-Kinder Joy lini-Modelator do K Joya,Nieokreślony
160650,PM,10143666,2014-04-01,1-Linia pakowania IFCR,Nieokreślony
160652,PM,10143508,2014-04-01,1-Tanks,Nieokreślony


In [146]:
import json, re
from pathlib import Path

def hierarchy_to_json(text: str,
                       out_path: str = "hierarchy.json",
                       key_re=r"(\d{6,8})|PLPA-[\w\-]+",
                       value_re=r"[A-ZŻŹĆĄŚĘŁÓŃ0-9].*?(\s{2,}|$)",
                       strip_columns=True):

    lines = text.splitlines()

    def indent_width(line):
        trimmed = re.sub(r"^[\s\|\-\+]*", "", line)
        return len(line) - len(trimmed)

    root = {}                 # ---- fixed: create first
    stack = [(-1, root)]      # ---- and then put on the stack

    for raw in lines:
        if not raw.strip():
            continue
        indent = indent_width(raw)

        key_match = re.search(key_re, raw)
        if not key_match:
            continue
        key = key_match.group(0)

        value_part = raw.split(key, 1)[-1]
        value_match = re.search(value_re, value_part)
        value = value_match.group(0).strip() if value_match else ""

        if strip_columns:
            value = re.sub(r"\s{2,}", " ", value)

        while stack and indent <= stack[-1][0]:
            stack.pop()

        parent = stack[-1][1]
        parent[key] = {"description": value, "children": {}}
        stack.append((indent, parent[key]["children"]))

    def squash(node):
        """
        Recursively turn the internal representation into the final JSON structure,
        retaining both the description and any children.
        """
        out = {"_desc": node["description"]}
        if node["children"]:
            out["children"] = {k: squash(v) for k, v in node["children"].items()}
        return out


    json_tree = {k: squash(v) for k, v in root.items()}

    Path(out_path).write_text(
        json.dumps(json_tree, ensure_ascii=False, indent=2), encoding="utf-8"
    )
    print(f"JSON saved → {out_path}")
    return json_tree

with open("urządzenie_struktura.txt", encoding="utf-8") as f:
    txt = f.read()

# nested = hierarchy_to_json(txt, "urządzenia_struktura.json")

In [147]:
import pandas as pd, json, re

df = pd.read_csv("cleaned_lista_zawiadomień.csv", encoding="utf-8")
with open("urządzenia_struktura.json", encoding="utf-8") as f:
    tree = json.load(f)


def flatten(node, *, parent_1_desc=None,
            linia_ogolna=None,  linia_ogolna_desc=None,
            linia_szczeg=None, linia_szczeg_desc=None):
    result = {}

    for key, value in node.items():
        desc = value.get("_desc", "").strip()

        # ── production line codes & names ─────────────────────────
        if '-' in key:
            dash_cnt = key.count('-')
            if dash_cnt == 3:                       # e.g. PLPA-PR-U11-010703
                linia_ogolna      = key
                linia_ogolna_desc = desc
            elif dash_cnt == 4:                     # e.g. PLPA-PR-U11-010703-001
                linia_szczeg      = key
                linia_szczeg_desc = desc

        # ── top-level “1- …” asset ───────────────────────────────
        if desc.startswith("1-"):
            parent_1_desc = desc

        # store current node
        if desc:
            result[desc] = {
                "ozn": parent_1_desc,
                "linia_ogolna":      linia_ogolna,
                "linia_ogolna_desc": linia_ogolna_desc,
                "linia_szczeg":      linia_szczeg,
                "linia_szczeg_desc": linia_szczeg_desc,
                "id": key
            }

        # recurse
        children = value.get("children", {})
        result.update(
            flatten(children,
                    parent_1_desc=parent_1_desc,
                    linia_ogolna=linia_ogolna,      linia_ogolna_desc=linia_ogolna_desc,
                    linia_szczeg=linia_szczeg,      linia_szczeg_desc=linia_szczeg_desc)
        )
    return result

lookup = flatten(tree)

cols = ["First Level Ozn",
        "Linia ogólna", "Linia ogólna nazwa",
        "Linia szczegółowa", "Linia szczegółowa nazwa"]

def match(row):
    m = lookup.get(str(row["Oznaczenie"]).strip())
    if m:
        return pd.Series([
            m["ozn"],
            m["linia_ogolna"],      m["linia_ogolna_desc"],
            m["linia_szczeg"],      m["linia_szczeg_desc"]
        ])
    return pd.Series([pd.NA] * len(cols))

df[cols] = df.apply(match, axis=1)


In [148]:
df = df.dropna(subset=cols, how="all").reset_index(drop=True)
df.to_csv("zgloszenia_with_hierarchy.csv", index=False)

In [149]:
na_counts = df[cols].isna().sum()        
print(na_counts)

First Level Ozn             8195
Linia ogólna                   0
Linia ogólna nazwa             0
Linia szczegółowa          18174
Linia szczegółowa nazwa    18174
dtype: int64


In [150]:
unique_counts = df[cols].nunique(dropna=True) 
print(unique_counts)

First Level Ozn            1807
Linia ogólna                 81
Linia ogólna nazwa           81
Linia szczegółowa           106
Linia szczegółowa nazwa     106
dtype: int64


In [151]:
df

,Rodzaj zawiad.,Zawiadomienie,Data zawiadom.,Oznaczenie,Priorytet,First Level Ozn,Linia ogólna,Linia ogólna nazwa,Linia szczegółowa,Linia szczegółowa nazwa
0,1P,12708097,2025-06-30,1-Zespół pomp próżniowych BUSCH,4-niski,1-Zespół pomp próżniowych BUSCH,PLPA-PR-U12-010801,KINDER CARDS,PLPA-PR-U12-010801-002,KINDER CARDS MODELATOR
1,1P,12707957,2025-06-30,1-Trans. Sandwick za walcami 64M,Nieokreślony,1-Trans. Sandwick za walcami 64M,PLPA-PR-U32-012110,Prep2 Krem KB,None,None
2,1P,12707755,2025-06-30,1-Zbiornik olejowy 1m^3,Nieokreślony,1-Zbiornik olejowy 1m^3,PLPA-PR-U32-012211,Ciemny krem (K CARDS/KJ),None,None
3,1P,12707745,2025-06-30,1-Piec wafla górnego KB2,Nieokreślony,1-Piec wafla górnego KB2,PLPA-PR-U43-011445,KB 2.,PLPA-PR-U43-011445-001,KB 2. Piece z nawilżaczami
4,PT,12707720,2025-06-30,2-Dozownik obrotowo - łopatkowy,3-średni,1-Silos kompletny odpadu waflowego,PLPA-PR-U12-011312,NUTELLA B-READY,PLPA-PR-U12-011312-002,NUTELLA B-READY MODELATOR
...,...,...,...,...,...,...,...,...,...,...
123450,PM,10143837,2014-04-02,1-Łukowy przechładzacz wafla 1 RAF ARKA,Nieokreślony,1-Łukowy przechładzacz wafla 1 RAF ARKA,PLPA-PR-U11-010703,RAFFAELLO LINIA 2,PLPA-PR-U11-010703-001,PIECE Z NAWILŻACZAMI RAFFAELLO
123451,PM,10143754,2014-04-01,1- Maszyna termoformująca TFT2,Nieokreślony,1- Maszyna termoformująca TFT2,PLPA-PR-U22-019329,KJ TERMOFORMATURA,PLPA-PR-U22-019329-001,KJ TERMOFORMOWANIE
123452,PM,10143739,2014-04-01,1-Piec do wypieku wafla RAF 1,Nieokreślony,1-Piec do wypieku wafla RAF 1,PLPA-PR-U11-010703,RAFFAELLO LINIA 2,PLPA-PR-U11-010703-001,PIECE Z NAWILŻACZAMI RAFFAELLO
123453,PM,10143720,2014-04-01,1-Drukarki,Nieokreślony,1-Drukarki,PLPA-PR-U43-011444,KB / KBW Isola Multipack,None,None


In [ ]:
import pandas as pd
import plotly.graph_objects as go
from pathlib import Path

DATA_PATH = Path("zgloszenia_with_hierarchy.csv")   
df = (
    pd.read_csv(DATA_PATH)
      .assign(Data_zawiadom=pd.to_datetime(
              pd.to_datetime(df["Data zawiadom."], errors="coerce").dt.date))
)

df = df.rename(columns={
    "Linia szczegółowa nazwa": "Linia_nazwa_szczeg",
    "Linia ogólna nazwa":      "Linia_nazwa_ogólna"
})

df["YearMonth"] = df["Data_zawiadom"].dt.to_period("M").astype(str)
metric = (
    df.groupby(["Linia_nazwa_szczeg", "YearMonth"])
      .size().rename("Count")
      .reset_index()
)

def make_trace(line_name):
    tmp = metric.loc[metric["Linia_nazwa_szczeg"].eq(line_name)]
    return go.Bar(
        x=tmp["YearMonth"],
        y=tmp["Count"],
        name=line_name,
        visible=False,     
    )

all_lines = sorted(metric["Linia_nazwa_szczeg"].dropna().unique())

traces = [make_trace(l) for l in all_lines]
traces[0].visible = True

buttons = [
    dict(
        label=line,
        method="update",
        args=[{"visible": [i == k for i in range(len(traces))]},
              {"title": f"Liczba zgłoszeń – {line}"}]
    )
    for k, line in enumerate(all_lines)
]

fig = go.Figure(
    data=traces,
    layout=dict(
        title=f"Liczba zgłoszeń – {all_lines[0]}",
        xaxis_title="Rok-miesiąc",
        yaxis_title="Liczba zgłoszeń",
        updatemenus=[dict(
            active=0,
            buttons=buttons,
            bgcolor="white"
        )]
    )
)
fig.update_layout(barmode="group", height=500, width=950)
fig.show()